In [1]:
from matplotlib import pyplot as plt
import psycopg2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Paramètres de connexion à la base de données PostgreSQL
conn_params = {
    "host": "localhost",
    "port": "15432", 
    "database": "postgres",
    "user": "admin",
    "password": "admin"
}

# Connexion à la base de données
conn = psycopg2.connect(**conn_params)

# Création d'un curseur pour exécuter des requêtes SQL
cur = conn.cursor()

# Requête SQL pour sélectionner les données de la table économie
query_economie = "SELECT * FROM economie;"
cur.execute(query_economie)
rows_economie = cur.fetchall()

# Requête SQL pour sélectionner les données du premier tour (ou du second tour si nécessaire)
query_election = "SELECT code_postal, \"% Voix/Ins 1\", \"% Voix/Ins 2\" FROM election_2022_t2"
cur.execute(query_election)
rows_election = cur.fetchall()

# Fermeture du curseur et de la connexion à la base de données
cur.close()
conn.close()

# Création des DataFrames à partir des résultats des requêtes
df_economie = pd.DataFrame(rows_economie, columns=['avg_1982', 'avg_1983', 'avg_1984', 'avg_1985', 'avg_1986', 'avg_1987', 'avg_1988', 'avg_1989', 'avg_1990', 'avg_1991', 'avg_1992', 'avg_1993', 'avg_1994', 'avg_1995', 'avg_1996', 'avg_1997', 'avg_1998', 'avg_1999', 'avg_2000', 'avg_2001', 'avg_2002', 'avg_2003', 'avg_2004', 'avg_2005', 'avg_2006', 'avg_2007', 'avg_2008', 'avg_2009', 'avg_2010', 'avg_2011', 'avg_2012', 'avg_2013', 'avg_2014', 'avg_2015', 'avg_2016', 'avg_2017', 'avg_2018', 'avg_2019', 'avg_2020', 'avg_2021', 'avg_2022', 'avg_2023', 'code_postal', 'Nb_de_ménages_fiscaux_2021', 'Médiane_du_niveau_de_vie_2021', 'Part_des_minima_sociaux_dans_le_rev_disp_2021', 'Part_des_prestations_familiales_dans_le_rev_disp_2021', 'Part_des_prestations_sociales_dans_le_rev_disp_2021', 'Part_des_prestations_logement_dans_le_rev_disp_2021', 'Part_des_indemnités_de_chômage_dans_le_rev_disp_2021', '2022', '2005', '2000', 'id_economie'])
df_election = pd.DataFrame(rows_election, columns=['code_postal', '% Voix/Ins 1', '% Voix/Ins 2'])

# Supprimer les colonnes non désirées dans df_economie
df_economie.drop(['id_economie'], axis=1, inplace=True)

# Fusionner les données économiques avec les données de l'élection sur la colonne code_postal
df_merged = pd.merge(df_economie, df_election, on='code_postal')

# Sélectionner uniquement les données pour La République En Marche
df_party = df_merged

# Séparer les features (X) et la target (y)
X = df_party.drop(['code_postal', '% Voix/Ins 1', '% Voix/Ins 2'], axis=1)
y = df_party['% Voix/Ins 1']

# Diviser les données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle de régression linéaire sur l'ensemble d'entraînement
model = LinearRegression()
model.fit(X_train, y_train)

# Prédire les résultats sur l'ensemble de test pour évaluer la performance du modèle
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Erreur quadratique moyenne sur l'ensemble de test:", mse)

# Utiliser le modèle entraîné pour prédire le vote de La République En Marche dans le département 89
dept_89_economie = df_economie[df_economie['code_postal'] == 89].drop('code_postal', axis=1)
prediction_dept_89 = model.predict(dept_89_economie)
print("Prédiction du vote de La République En Marche dans le département 89:", prediction_dept_89[0])


Erreur quadratique moyenne sur l'ensemble de test: 64.96468260708079
Prédiction du vote de La République En Marche dans le département 89: 31.58077046079928
